In [1]:
!pip install minsearch



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
!pip install groq



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
import requests 
from minsearch import AppendableIndex
import json
import chat_assistant
from groq import Groq
import os, re

In [4]:
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
index = AppendableIndex(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [7]:
index.search('Can i still join the course?')

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Here’s how you join a in Slack: https://slack.com/

In [8]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [9]:
search('Can i still join the course?')

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  '_id': 2},
 {'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.",
  'section': 'General course-related questions',
  'question': 'Certificate - Can I follow the course in a self-paced mode and get a certificate?',
  'course': 'data-engineering-zoomcamp',
  '_id': 11},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the cou

In [10]:
question = 'Can I still join the course?'


In [11]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

<QUESTION>
{question}
</QUESTION>

<CONTEXT>
{context}
</CONTEXT>
""".strip()

def build_prompt(query, search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
search_results = search(question)

In [13]:
prompt = build_prompt(question, search_results)

In [14]:
# Set up the Groq client using your API key
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Define the LLM call using Groq's chat completion
def llm(prompt):
    response = client.chat.completions.create(
        model="llama3-70b-8192",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

In [15]:
answer = llm(prompt)

In [16]:
print(answer)

Based on the provided context, the answer to the question "Can I still join the course?" is:

Yes, you can still join the course. According to the context, registration is not required to participate in the course, and you can start learning and submitting homeworks without registering.


In [17]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [18]:
rag("How do I patch KDE under FreeBSD?")

'According to our FAQ database, you can patch KDE under FreeBSD by using the following command:\n\n`portmaster -a`\n\nThis command will update and patch KDE and its dependencies using the portmaster utility.\n\nPlease note that this answer is specific to the FreeBSD operating system and assumes you have the portmaster utility installed.'

In [19]:
print(llm("How do I patch KDE under FreeBSD?"))

Patching KDE under FreeBSD involves updating the KDE ports tree and rebuilding the KDE packages. Here's a step-by-step guide to help you do so:

**Prerequisites:**

1. Make sure you have the `portsnap` utility installed and configured. If not, install it by running `pkg install portsnap`.
2. Update your ports tree by running `portsnap fetch update`.
3. Ensure you have the necessary build dependencies installed, such as `cmake`, `gcc`, and `perl`. You can check the dependencies by running `make config` in the KDE port directory (e.g., `/usr/ports/x11/kde-applications`).

**Patching KDE:**

1. **Update the KDE ports tree:** Run `portmaster -r kde` (or `portupgrade -r kde` if you're using `portupgrade` instead of `portmaster`) to update the KDE ports tree. This command will fetch the latest KDE ports and dependencies.
2. **Rebuild KDE:** Run `portmaster kde` (or `portupgrade kde`) to rebuild KDE and its dependencies. This may take some time, depending on your system's resources and the nu

In [20]:
rag("How do I run Kafka in Docker?")

'According to the provided context, to run Kafka in Docker, you need to start all the instances using `docker compose up -d` in the docker compose yaml file folder. This is mentioned in the answer to the question "kafka.errors.NoBrokersAvailable: NoBrokersAvailable" in the "Module 6: streaming with kafka" section.'

#### Agentic RAG

In [21]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
{question}
</QUESTION>

<CONTEXT> 
{context}
</CONTEXT>

If CONTEXT is EMPTY, you can use our FAQ database.
In this case, use the following output template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>"
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}
""".strip()

In [22]:
question = 'Can I still join the course?'
context = 'EMPTY'

In [23]:
prompt = prompt_template.format(question=question, context=context)


In [24]:
answer_json = llm(prompt)

In [25]:
print(answer)

Based on the provided context, the answer to the question "Can I still join the course?" is:

Yes, you can still join the course. According to the context, registration is not required to participate in the course, and you can start learning and submitting homeworks without registering.


In [26]:
def parse_json(answer_json):
    # Remove Markdown code fencing and whitespace
    cleaned = answer_json.strip().removeprefix("```json").removesuffix("```").strip()
    return json.loads(cleaned)

# Example usage
#answer = parse_json(answer_json)

In [27]:
answer

'Based on the provided context, the answer to the question "Can I still join the course?" is:\n\nYes, you can still join the course. According to the context, registration is not required to participate in the course, and you can start learning and submitting homeworks without registering.'

In [28]:
def build_context(search_results):
    context = ""

    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    return context.strip()

In [29]:
search_results = search(question)
context = build_context(search_results)
prompt = prompt_template.format(question=question, context=context)
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.
At the beginning the context is EMPTY.

<QUESTION>
Can I still join the course?
</QUESTION>

<CONTEXT> 
section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Certificate - Can I follow the course in a self-paced mode and get a certificate?
answer: No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the cour

In [30]:
answer_json = llm(prompt)


In [31]:
print(answer_json)

{
"action": "ANSWER",
"answer": "Yes, you can still join the course after the start date. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
"source": "CONTEXT"
}


#### Agentic Search

In [32]:
prompt_template = """
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than {max_iterations} iterations for a given student question.
The current iteration number: {iteration_number}. If we exceed the allowed number 
of iterations, give the best possible answer with the provided information.

Output templates:

If you want to perform search, use this template:

{{
"action": "SEARCH",
"reasoning": "<add your reasoning here>",
"keywords": ["search query 1", "search query 2", ...]
}}

If you can answer the QUESTION using CONTEXT, use this template:

{{
"action": "ANSWER_CONTEXT",
"answer": "<your answer>",
"source": "CONTEXT"
}}

If the context doesn't contain the answer, use your own knowledge to answer the question

{{
"action": "ANSWER",
"answer": "<your answer>",
"source": "OWN_KNOWLEDGE"
}}

<QUESTION>
{question}
</QUESTION>

<SEARCH_QUERIES>
{search_queries}
</SEARCH_QUERIES>

<CONTEXT> 
{context}
</CONTEXT>

<PREVIOUS_ACTIONS>
{previous_actions}
</PREVIOUS_ACTIONS>
""".strip()

In [33]:
question = 'how do I do well on module 1'
max_iterations = 3
iteration_number = 0
search_queries = []
search_results  = []
previous_actions = []

In [34]:
context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [35]:
answer = llm(prompt)


In [36]:
print(answer)

{
"action": "SEARCH",
"reasoning": "To provide guidance on how to do well on module 1, I need to gather information on the module's content, learning objectives, and recommended study strategies.",
"keywords": ["module 1 learning objectives", "module 1 study guide", "tips for success in module 1"]
}


In [37]:
answer = parse_json(answer)

In [38]:
answer

{'action': 'SEARCH',
 'reasoning': "To provide guidance on how to do well on module 1, I need to gather information on the module's content, learning objectives, and recommended study strategies.",
 'keywords': ['module 1 learning objectives',
  'module 1 study guide',
  'tips for success in module 1']}

In [39]:
keywords = answer['keywords']


In [40]:
for kw in keywords:
    search_queries.append(kw)
    sr = search(kw)
    search_results.extend(sr)

In [41]:
len(search_results)

15

In [42]:
def dedup(seq):
    seen = set()
    result = []
    for el in seq:
        _id = el['_id']
        if _id in seen:
            continue
        seen.add(_id)
        result.append(el)
    return result

In [43]:
search_results = dedup(search_results)

In [44]:
len(search_results)

6

In [45]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [46]:
previous_actions.append(answer)

In [47]:
iteration_number = 2

context = build_context(search_results)

prompt = prompt_template.format(
    question=question,
    context=context,
    search_queries="\n".join(search_queries),
    previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
    max_iterations=max_iterations,
    iteration_number=iteration_number
)

In [48]:
answer_json = llm(prompt)

In [49]:
print(prompt)

You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current iteration number

In [50]:
print(answer_json)


{
"action": "SEARCH",
"reasoning": "To provide more specific guidance on how to do well on module 1, I need to gather information on common challenges and pitfalls in module 1, as well as successful strategies and best practices.",
"keywords": ["common mistakes in module 1", "module 1 pitfalls", "module 1 best practices", "success stories in module 1"]
}


In [51]:
question = "what do I need to do to be successful at module 1?"

search_queries = []
search_results = []
previous_actions = []

iteration = 0

while True:
    print(f'ITERATION #{iteration}...')

    context = build_context(search_results)
    prompt = prompt_template.format(
        question=question,
        context=context,
        search_queries="\n".join(search_queries),
        previous_actions='\n'.join([json.dumps(a) for a in previous_actions]),
        max_iterations=3,
        iteration_number=iteration
    )

    print(prompt)

    answer_json = llm(prompt)
    answer = parse_json(answer_json)
    print(json.dumps(answer, indent=2))

    previous_actions.append(answer)

    action = answer['action']
    if action != 'SEARCH':
        break

    keywords = answer['keywords']
    search_queries = list(set(search_queries) | set(keywords))
    
    for k in keywords:
        res = search(k)
        search_results.extend(res)

    search_results = dedup(search_results)
    
    iteration = iteration + 1
    if iteration >= 4:
        break

    print()

ITERATION #0...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefully analyze the CONTEXT and generate the requests to deeply explore the topic. 

Don't use search queries used at the previous iterations.

Don't repeat previously performed actions.

Don't perform more than 3 iterations for a given student question.
The current 

{
  "action": "SEARCH",
  "reasoning": "To answer the question, I need to gather more information about the requirements for success in Module 1.",
  "keywords": [
    "module 1 requirements",
    "success factors in module 1",
    "module 1 expectations"
  ]
}

ITERATION #1...
You're a course teaching assistant.

You're given a QUESTION from a course student and that you need to answer with your own knowledge and provided CONTEXT.

The CONTEXT is build with the documents from our FAQ database.
SEARCH_QUERIES contains the queries that were used to retrieve the documents
from FAQ to and add them to the context.
PREVIOUS_ACTIONS contains the actions you already performed.

At the beginning the CONTEXT is empty.

You can perform the following actions:

- Search in the FAQ database to get more data for the CONTEXT
- Answer the question using the CONTEXT
- Answer the question using your own knowledge

For the SEARCH action, build search requests based on the CONTEXT and the QUESTION.
Carefu

In [52]:
answer

{'action': 'ANSWER_CONTEXT',
 'answer': 'Unfortunately, the provided context does not contain specific information about the requirements for success in Module 1. However, based on the context, it seems that success in the modules is often dependent on proper installation and configuration of the required tools and environments, such as virtual environments, Docker, and Spark. Additionally, it is important to pay attention to module-specific requirements and configurations. I recommend carefully following the instructions and guidelines provided for each module and seeking help when needed.',
 'source': 'CONTEXT'}

In [53]:
iteration

3

#### Function calling ("tool use")

In [54]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5,
        output_ids=True
    )

    return results

In [55]:
search_tool = [
    {
        "type": "function",
        "function": {
            "name": "search",
            "description": "Search the FAQ database",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "Search query text to look up in the course FAQ."
                    }
                },
                "required": ["query"],
                "additionalProperties": False
            }
        }
    }
]

In [56]:
def do_call(tool_call_response):
    function_name = tool_call_response.function.name
    arguments = json.loads(tool_call_response.function.arguments)


    f = globals()[function_name]
    result = f(**arguments)

    return {
        "type": "function_call_output",
        "call_id": tool_call_response.id,
        "output": json.dumps(result, indent=2),
    }

In [57]:

question = "How do I do well in module 1?"

developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.
If you look up something in FAQ, convert the student question into multiple queries.
""".strip()

tools = search_tool

chat_messages = [
    {"role": "system", "content": developer_prompt},
    {"role": "user", "content": question}
]

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",  # enables built-in search/code tools :contentReference[oaicite:5]{index=5}
    messages=chat_messages,
    tools=tools,
    tool_choice="auto"
)

# 3. Extract tool call
choice = response.choices[0]

if choice.message.tool_calls:
    tool_call = choice.message.tool_calls[0]
    print("🔧 Tool Call Detected:")
    print("Name:", tool_call.function.name)
    print("Arguments:", tool_call.function.arguments)
    print("Call ID:", tool_call.id)
    #print("Status:", tool_call.status)
else:
    print("💬 Response:", choice.message.content)

🔧 Tool Call Detected:
Name: search
Arguments: {"query":"module 1 tips"}
Call ID: zwayjr9eq


In [58]:
calls = response.choices[0].message.tool_calls

In [59]:
calls

[ChatCompletionMessageToolCall(id='zwayjr9eq', function=Function(arguments='{"query":"module 1 tips"}', name='search'), type='function'),
 ChatCompletionMessageToolCall(id='deys5gt85', function=Function(arguments='{"query":"module 1 requirements"}', name='search'), type='function'),
 ChatCompletionMessageToolCall(id='7pzd0311c', function=Function(arguments='{"query":"module 1 assessment"}', name='search'), type='function')]

In [60]:
import pprint
print("➡️ chat_messages snapshot:")
pprint.pprint(chat_messages)


➡️ chat_messages snapshot:
[{'content': "You're a course teaching assistant. \n"
             "You're given a question from a course student and your task is "
             'to answer it.\n'
             'If you look up something in FAQ, convert the student question '
             'into multiple queries.',
  'role': 'system'},
 {'content': 'How do I do well in module 1?', 'role': 'user'}]


In [61]:
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=chat_messages,
    tools=tools,
    tool_choice="auto"
)

tool_calls = response.choices[0].message.tool_calls


In [62]:
for tool_call in tool_calls:
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    tool_result = search(**args)

    # Append assistant's tool_call
    chat_messages.append({
        "role": "assistant",
        "content": None,
        "function_call": {
            "name": name,
            "arguments": json.dumps(args)
        }
    })

    # Append tool's result
    chat_messages.append({
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps(tool_result)
    })


In [63]:
response = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=chat_messages
)

chat_messages.append({
    "role": "assistant",
    "content": response.choices[0].message.content
})

print(response.choices[0].message.content)


To do well in Module 1, it's essential to ensure that you have the correct versions of the required modules installed. You can encounter a `ModuleNotFoundError` if the versions of the modules you're using don't match the ones required by the course materials. 

Here are some specific tips that might help:

1. Verify the version of Py4J: Make sure the version of Py4J matches the one required by the course. You can check the version by running `ls ${SPARK_HOME}/python/lib/` and then update the export command accordingly.
2. Install the correct version of pyspark: If you're using pyspark, ensure that you've installed the correct version. You can try running `!pip3 install pyspark` instead of `!pip install pyspark` if you're encountering issues.
3. Install the required Python modules: Ensure that you've installed the necessary Python modules, such as `psycopg2-binary` and `pytz`. You can install them using pip or conda.
4. Update conda or pip: If you're encountering issues with module inst

In [64]:
chat_messages

[{'role': 'system',
  'content': "You're a course teaching assistant. \nYou're given a question from a course student and your task is to answer it.\nIf you look up something in FAQ, convert the student question into multiple queries."},
 {'role': 'user', 'content': 'How do I do well in module 1?'},
 {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'search',
   'arguments': '{"query": "Module 1 success tips"}'}},
 {'role': 'tool',
  'tool_call_id': 'gvx6gvxrd',
  'content': '[{"text": "Even after installing pyspark correctly on linux machine (VM ) as per course instructions, faced a module not found error in jupyter notebook .\\nThe solution which worked for me(use following in jupyter notebook) :\\n!pip install findspark\\nimport findspark\\nfindspark.init()\\nThereafter , import pyspark and create spark contex<<t as usual\\nNone of the solutions above worked for me till I ran !pip3 install pyspark instead !pip install pyspark.\\nFilter based on conditions based on

In [65]:
developer_prompt = """
You are a helpful course teaching assistant.

You receive a question from a student and your goal is to provide a complete, helpful answer.

If you need extra information, search the FAQ using the provided tool. If one search is not enough, you may issue multiple searches.

But once you have gathered enough information, **you must summarize and answer the original student question** clearly.

After answering, **ask the student a thoughtful follow-up question** to keep the conversation going.
""".strip()

chat_messages = [
    {"role": "system", "content": developer_prompt},
]

In [66]:
def do_call_(function_name, **kwargs):
    if function_name == "search":
        return search(**kwargs)
    # Add more tools here if needed
    else:
        return {"error": f"Unknown tool: {function_name}"}

In [67]:
MAX_DEPTH = 5  # to prevent infinite loops
depth = 0

while True:  # main Q&A loop
    question = input("\n❓ Ask your course question (or type 'stop'): ")
    if question.lower() == 'stop':
        break

    chat_messages.append({"role": "user", "content": question})

    depth = 0
    while depth < MAX_DEPTH:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=chat_messages,
            tools=tools,
        )

        has_message = False

        for choice in response.choices:
            msg = choice.message

            # Tool call
            if msg.tool_calls:
                for tool_call in msg.tool_calls:
                    print(f"\n🔧 Tool call → {tool_call.function.name}: {tool_call.function.arguments}\n")
                    args = json.loads(tool_call.function.arguments)
                    result = do_call_(tool_call.function.name, **args)

                    # Append assistant tool call + tool result
                    chat_messages.append({
                        "role": "assistant",
                        "content": None,
                        "function_call": {
                            "name": tool_call.function.name,
                            "arguments": json.dumps(args)
                        }
                    })
                    chat_messages.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(result)
                    })

            # Final assistant message
            if msg.content:
                print(f"\n🤖 Assistant: {msg.content}")
                chat_messages.append({"role": "assistant", "content": msg.content})
                has_message = True

        if has_message:
            break

        depth += 1
    else:
        print("⚠️ Reached tool recursion limit without assistant reply.")


🔧 Tool call → search: {"query":"Module 1 requirements and tips for success"}


🤖 Assistant: To do well in Module 1, it is essential to understand the fundamentals of Docker and Terraform. Based on the search results, it appears that some students have encountered issues with module not found errors, which can be resolved by installing the required packages or updating the export command.

Here are some tips for success in Module 1:

1. Make sure to install the required packages, such as pyspark, and update the export command if necessary.
2. Use the correct version of py4j to avoid ModuleNotFoundError.
3. Create a virtual environment and run requirements.txt and the python files in that environment.
4. Ensure that the Docker images are up and running before attempting to run the python files.

By following these tips and understanding the concepts of Docker and Terraform, you can set yourself up for success in Module 1.

What specific topics or concepts in Module 1 are you struggling 

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jxwjrv3bec889xfpmd5yftva` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99796, Requested 2130. Please try again in 27m44.001s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

#### Multiple tools

In [68]:
def add_entry(question, answer):
    doc = {
        'question': question,
        'text': answer,
        'section': 'user added',
        'course': 'data-engineering-zoomcamp'
    }
    index.append(doc)

In [69]:
add_entry_description = {
    "type": "function",
    "function": {
        "name": "add_entry",
        "description": "Add an entry to the FAQ database.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {
                    "type": "string",
                    "description": "The question to be added to the FAQ database."
                },
                "answer": {
                    "type": "string",
                    "description": "The answer to the question."
                }
            },
            "required": ["question", "answer"],
            "additionalProperties": False
        }
    }
}


In [70]:
import chat_assistant

tools = chat_assistant.Tools()

tools.add_tool(search, search_tool)

TypeError: list indices must be integers or slices, not str

In [71]:
tools.add_tool(add_entry, add_entry_description["function"])


In [72]:
tools.get_tools()


[{'type': 'function',
  'function': {'name': 'add_entry',
   'description': 'Add an entry to the FAQ database.',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': 'The question to be added to the FAQ database.'},
     'answer': {'type': 'string',
      'description': 'The answer to the question.'}},
    'required': ['question', 'answer'],
    'additionalProperties': False}}}]

In [73]:
developer_prompt = """
You're a course teaching assistant. 
You're given a question from a course student and your task is to answer it.

Use FAQ if your own knowledge is not sufficient to answer the question.

At the end of each response, ask the user a follow up question based on your answer.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [74]:
chat.run()

AttributeError: 'ChatCompletionMessageToolCall' object has no attribute 'call_id'

In [75]:
index.docs[-1]


{'question': 'How do I do well in Module 1?',
 'text': 'To do well in Module 1, read and understand the course syllabus, attend classes and participate in discussions, complete all assignments and quizzes, review and practice regularly, and seek help when needed.',
 'section': 'user added',
 'course': 'data-engineering-zoomcamp'}